In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [25]:
import sys; sys.path.append('../slip_rate_tools/')

#import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import slip_rate_tools as srt
import numpy as np
from importlib import reload

In [51]:
reload(srt)

<module 'slip_rate_tools' from '../slip_rate_tools/slip_rate_tools.py'>

In [3]:
from numba import jit

In [4]:
'''
Test data.  This will be removed when the adding data interface is built.
'''

offset_df = pd.read_csv('../test_data/offsets.csv')
offset_df['offset_m'] = offset_df.offset_in * 200.

t1 = offset_df[offset_df.unit == 'T1']
qa = offset_df[offset_df.unit == 'Qa']
qao = offset_df[offset_df.unit == 'Qao']

#qa['offset_m'] += 200.

t1_age = {'mean': 24., 'sd':8.}
qa_age = {'mean': 50., 'sd':20.}
qao_age = {'mean':100., 'sd':32.}

#qao_age['mean'] += 200

T1 = srt.OffsetMarker(age_mean=t1_age['mean'], age_sd=t1_age['sd'],
                      offset_vals=t1.offset_m, offset_probs=t1.rel_prob)

Qa = srt.OffsetMarker(age_mean=qa_age['mean'], age_sd=qa_age['sd'],
                      offset_vals=qa.offset_m, offset_probs=qa.rel_prob)

Qao = srt.OffsetMarker(age_mean=qao_age['mean'], age_sd=qao_age['sd'],
                      offset_vals=qao.offset_m, offset_probs=qao.rel_prob)


In [54]:
def run_linear_interp(offset_list, n_iters, zero_offset_age=0.,
                      check_increasing=False, check_rate_change=False):
    '''
    Main linear interpolation function. Runs both 

    Arguments:
    offset_list = list of OffsetMarkers, in increasing age order.
    n_iters = integer specifying number of Monte Carlo iterations.
    zero_offset_age = float specifying the age at which no more offset occurs;
                      i.e. the last age of faulting.
    check_increasing = Boolean value to ensure increasing offsets with
                       increasing ages.
    check_rate_change = Boolean value to perform a piecewise linear
                        interpolation (2 pieces) 
    '''

    srt.check_unit_consistency(offset_list)

    n_pts = len(offset_list) + 1 # accounting for 0 offset

    print('sampling offset markers')
    age_arr, off_arr = srt.make_age_offset_arrays(offset_list, n_iters,
                                             check_increasing=check_increasing)

    print('doing fits')
    results_df = do_linear_fits(age_arr, off_arr, 
                                    check_rate_change=check_rate_change,
                                    trim_results=True)

    results_df['log_like_1'] = srt.log_likelihood(results_df.sumsq1, n_pts)

    if check_rate_change==True:
        num_1_odds = srt.rate_change_test(results_df, 4, print_res=True)


    elif check_rate_change==False:
        print('\nbest fit slip rate results:')
        print(results_df.m.describe())

    return results_df, age_arr

In [56]:
%%timeit
res_df, age_arr= run_linear_interp([T1, Qa, Qao], 500, 
                                       check_increasing=True, 
                                       check_rate_change=False)

sampling offset markers
doing fits

best fit slip rate results:
count    500.000000
mean       2.152147
std        1.080795
min        0.547343
25%        1.319981
50%        1.920500
75%        2.762681
max        6.808655
Name: m, dtype: float64
sampling offset markers
doing fits

best fit slip rate results:
count    500.000000
mean       2.116950
std        1.122299
min        0.514049
25%        1.200694
50%        1.883910
75%        2.809235
max        5.919213
Name: m, dtype: float64
sampling offset markers
doing fits

best fit slip rate results:
count    500.000000
mean       2.063494
std        1.061937
min        0.495486
25%        1.174829
50%        1.800766
75%        2.703643
max        8.497009
Name: m, dtype: float64
sampling offset markers
doing fits

best fit slip rate results:
count    500.000000
mean       2.165667
std        1.181048
min        0.535149
25%        1.190322
50%        1.882459
75%        2.904536
max        7.293999
Name: m, dtype: float64
1 loops,

In [53]:
do_linear_fits = jit(srt.do_linear_fits)

In [7]:
%%timeit
rate_hist_df = srt.make_rate_hist_array(res_df, age_arr, n_segments=2)

1 loops, best of 3: 13.5 s per loop


In [45]:
%%timeit
rate_hist_df = srt.make_rate_hist_array(res_df, age_arr, n_segments=2)

1 loops, best of 3: 14.6 s per loop


In [9]:
make_rate_hist_array_jit = jit(srt.make_rate_hist_array)

In [11]:
%%timeit
rate_hist_df2 = make_rate_hist_array_jit(res_df, age_arr, n_segments=2)

1 loops, best of 3: 13.5 s per loop


In [16]:
#%%timeit
#srt.piecewise_rate_interp(1.1, 3.3, 96., 136., 160.)

1000 loops, best of 3: 535 µs per loop


In [29]:
%%timeit
srt.piecewise_rate_interp(1.1, 3.3, 96., 136., 160.)

10000 loops, best of 3: 43 µs per loop


In [23]:
def piecewise_rate_interp(rate1, rate2, breakpt, run_time_max, sim_time_max,
                          zero_offset_age=0., num_pts=1000):



    times = np.linspace(zero_offset_age, sim_time_max, num_pts)
    #slip_rate_history = pd.Series(index=times, data=np.zeros(num_pts))
    slip_rate_history = np.zeros(num_pts)
    
    
    zero_offset_idx = find_nearest_index_jit(times, zero_offset_age)
    run_time_max_idx = find_nearest_index_jit(times, run_time_max)
    breakpt_idx = find_nearest_index_jit(times, breakpt)
    
    slip_rate_history[zero_offset_idx : breakpt_idx] = rate1
    slip_rate_history[breakpt_idx : run_time_max_idx] = rate2
    

    #slip_rate_history.ix[zero_offset_age : breakpt] = rate1
    #slip_rate_history.ix[breakpt : run_time_max] = rate2
    
    

    return slip_rate_history

In [24]:
%%timeit
piecewise_rate_interp(1.1, 3.3, 96., 136., 160.)

The slowest run took 7647.98 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 30.9 µs per loop


In [14]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

def find_nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

@jit
def find_nearest_jit(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

@jit 
def find_nearest_index_jit(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx


In [50]:
age_arr[0,:]

array([   0.        ,   33.76030832,   46.21153838,  152.78650135])

In [ ]:
cum_hist_df = srt.make_cum_hist_array(rate_hist_df)